In [ ]:
import torch
import h5py
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
from deep_beamline_simulation.u_net import ImageProcessing

In [ ]:
"""
File path for train images
training images are the initial intensity of SRX beamline
the output is the output of the SRX beamline
"""
train_file = "../NSLS-II-CSX-1-beamline-rsOptExport/rsopt-srw-20220127150906/datasets/results.h5"
output_file = "../image_data/Intensity-At-Sample-63-3m.csv"


In [ ]:
"""
file path for test images
testing images are the initial intensity of CSX
the output is the output of the CSX beamline with varied aperture 
horizontal and vertical is 0.1
"""
test_file = "../image_data/initialInt_262.csv"
test_output_file = "../image_data/sample_555.csv"


In [ ]:
with h5py.File(train_file) as f:
    print(f.keys())
    beam_intensities = f["beamIntensities"]
    ip = ImageProcessing(beam_intensities)
    s = ip.smallest_image_size()
    print(s)

    print(beam_intensities[0])
    
    plt.figure()
    plt.imshow(beam_intensities[0], aspect="auto")
    plt.show()

    normalized_beam_intensities = (beam_intensities - np.mean(beam_intensities)) / np.std(beam_intensities)
    print(normalized_beam_intensities[0])
    
    resized_images = []
    for i in range(normalized_beam_intensities.shape[0]):
        resized_images.append(ip.resize(normalized_beam_intensities[i], height=139, length=41))

    plt.figure()
    plt.imshow(resized_images[0], aspect="auto")
    plt.show()
    
    print(len(resized_images))
    
    initial_beam_intensity = pd.read_csv("initial_beam_intensity_csx.csv", skiprows=1).to_numpy()
    normalized_initial_beam_intensity = ( initial_beam_intensity - np.mean(initial_beam_intensity) ) / np.std(initial_beam_intensity)
    resized_initial_beam_intensity = ip.resize(
        normalized_initial_beam_intensity,
        height=139,
        length=41
    )

    with h5py.File("preprocessed_results.h5", mode="w") as preprocessed_results:
        pi = preprocessed_results.create_dataset(
            "resized_initial_beam_intensity",
            (136, 40),
            data = resized_initial_beam_intensity
        )

        pbi = preprocessed_results.create_dataset(
            "preprocessed_beam_intensities",
            (100, 136, 40)
        )
        for i, resized_image in enumerate(resized_images):
            pbi[i] = resized_image
        
        param_vals = preprocessed_results.create_dataset_like("param_vals", f["paramVals"])
        for i, param_val in enumerate(f["paramVals"]):
            param_vals[i] = param_val

        params = preprocessed_results.create_dataset_like("params", f["params"])
        for i, param in enumerate(f["params"]):
            params[i] = param


In [ ]:
class BeamIntensityDataset:
    def __init__(self, beam_intensities, initial_beam_intensity, params, param_vals):
        self.beam_intensities = beam_intensities
        self.initial_beam_intensity = initial_beam_intensity
        self.params = params
        self.param_vals = param_vals
        
    def __getitem__(self, index):
        return self.beam_intensities[index], self.initial_beam_intensity, self.param_vals[i]
        
    def __len__(self):
        return self.beam_intensities.shape[0]
        

In [ ]:
data_file = h5py.File('preprocessed_results.h5')

In [ ]:
beamIntensities = np.array(data_file["preprocessed_beam_intensities"])

In [ ]:
paramVals = np.array(data_file["param_vals"])

In [ ]:
parameters = np.array(data_file['params'])

In [ ]:
initial_intensity = np.array(data_file['resized_initial_beam_intensity'])
plt.imshow(initial_intensity)
plt.show()

In [ ]:
bid = BeamIntensityDataset(beamIntensities, initial_intensity, parameters, paramVals)

In [ ]:
from torch.utils.data import DataLoader

In [ ]:
training_dataloader = DataLoader(bid, batch_size=10, shuffle=True)

In [ ]:
inten, initi, pvals = next(iter(training_dataloader))

In [ ]:
print(f"Intensity batch shape: {inten.size()}")
print(f"Param Values batch shape: {pvals.size()}")
print(f"Initial Intensity shape: {initi.size()}")

In [ ]:
int_img = inten[0].squeeze()
label = pvals[0]
plt.imshow(int_img, cmap="gray")
plt.show()
print(f"Label: {label}")

In [ ]:
init_img = initi[0].squeeze()
plt.imshow(init_img, cmap="gray")
plt.show()

In [ ]:
from deep_beamline_simulation.u_net import UNet

In [ ]:
model = UNet(136, 40)

In [ ]:
from torchinfo import summary

In [ ]:
summary(
    model,
    input_data=(torch.ones(10,1, 136, 40), [0.1,0.1]),
    col_names=("input_size", "output_size", "num_params")
)

In [ ]:
output_numpy = pd.read_csv(output_file, skiprows=1).to_numpy()
normalized_train_output_image = (output_numpy - np.mean(output_numpy)) / np.std(output_numpy)
resized_train_output_image = ip.resize(normalized_train_output_image[i], height=153, length=351)

In [ ]:
train_output_image = resized_train_output_image[None, None, :, :]

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_func = torch.nn.MSELoss()

In [ ]:
tb_loss = []

for e in range(0, 101):
    for i, data in enumerate(training_dataloader):
        train_inputs = data[0]
        train_parameters = data[1]
        
        train_inputs = train_inputs[:, None, :, :]
        
        predictions = model(train_inputs, train_parameters)
        crop_pred = predictions.detach()

        loss = loss_func(predictions, train_output_image)
        tb_loss.append(loss.data.numpy())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if e % 100 == 0:
        # output the loss
        print("Training Loss: " + str(loss.data.numpy()))
        print("Cropped Training Loss: " + str(cropped_train_loss.numpy()))